In [1]:
import pandas as pd
import numpy as np
import random

涉及到的变量：
| 变量名 | 说明 |
|:------|:-----|
| N ｜ 用户数量 ｜
| M ｜物品数量  ｜
| K | 全局约束，限制每个背包的资源分配，共计K个背包 |
| L | 局部约束，限制每个用户的消费 |
| x_{i,j} | 代表把物品j分配给用户i，增加p_{i,j}的收益，消耗b_{i,j,k}的资源 |
| p_{i,j} | 代表把物品j分配给用户i，增加p_{i,j}的收益 |
| b_{i,j,k} | 代表把物品j分配给用户i，消耗b_{i,j,k}的资源 |


假设：
1. 用户有100个，N = {0, 1, 2, 3, ... 99}
2. 物品数量10个，M = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
3. 全局约束，背包数量3个，K = 3，K代表的全局约束有3组，B_k有3个
4. 局部约束，每个用户的消费不能超过C_l，C_l有不超过100个；
5. b_{i,j,k}，有100 * 10 * 3 = 3000个

数据集包含：
1. b_consumption, b_{i,j,k}，3000个，代表用户i，分配物品j，消耗背包k的资源为b_{i,j,k}；
2. b_constraint, B_k，3个，代表背包k的资源限制为B_k；
3. C_l，30个，代表指定用户i对物品的分配不能超过C_l，C_l为整数；

---

```python
N = 100
M = 10
K = 3

b_consumption = np.zeros((N, M, K))
b_constraint = np.array([100, 121, 150])
c_constraint = np.zeros((N,))
p_profit = np.zeros((N, M))

for i in range(N):
    for j in range(M):
        for k in range(K):
            # 初始化: 用户i，分配物品j，消耗背包k的资源为b_{i,j,k}
            b_consumption[i][j][k] = random.randint(1, 10)  
    # 初始化：指定用户i对物品j的分配不能超过C_l
    c_constraint[i] = random.randint(30, 100)
    # 初始化，指定用户i对物品j的分配增加收益p_profit[i][j]
    p_profit[i][j] = random.randint(1, 20)
print(c_constraint)

# 保存数据到文件
np.save('data/b_consumption.npy', b_consumption)
np.save('data/b_constraint.npy', b_constraint)
np.save('data/c_constraint.npy', c_constraint)
np.save('data/p_profit.npy', p_profit)
```
---

In [43]:
N = 100
M = 10
K = 3

b_consumption = np.zeros((N, M, K))
b_constraint = np.array([100, 121, 150])
c_constraint = np.zeros((N,))
p_profit = np.zeros((N, M))

for i in range(N):
    for j in range(M):
        for k in range(K):
            # 初始化: 用户i，分配物品j，消耗背包k的资源为b_{i,j,k}
            b_consumption[i][j][k] = random.randint(1, 10)  
            
        # 初始化，指定用户i对物品j的分配增加收益p_profit[i][j]
        p_profit[i][j] = random.randint(1, 20)

    # 初始化：指定用户i对物品j的分配不能超过C_l
    c_constraint[i] = random.randint(30, 100)
print(c_constraint)

# 保存数据到文件
np.save('data/b_consumption.npy', b_consumption)
np.save('data/b_constraint.npy', b_constraint)
np.save('data/c_constraint.npy', c_constraint)
np.save('data/p_profit.npy', p_profit)

[ 73.  90.  59.  93.  62.  40.  75.  94.  82.  46.  92.  79.  38.  62.
  30.  85.  89.  68.  42.  77.  79.  97.  82.  84.  61.  48.  69.  52.
  76.  39.  83.  67.  34.  33.  48.  99.  72.  72.  62.  99.  66.  81.
  94.  37.  70.  85.  69.  43.  50. 100.  45.  43.  51.  60.  84.  60.
  51.  95.  56.  90.  91.  89.  74.  51.  76.  87.  45.  67.  71.  38.
  85.  82.  46.  34.  36.  50.  75.  70.  57. 100.  63.  55.  89.  96.
  36.  82.  93.  84. 100.  90.  77.  80.  33.  52.  86.  48.  36.  42.
  75.  38.]


In [44]:
# 加载数据
b_consumption = np.load('data/b_consumption.npy')
b_constraint = np.load('data/b_constraint.npy')
c_constraint = np.load('data/c_constraint.npy')
p_profit = np.load('data/p_profit.npy')

print(c_constraint)

[ 73.  90.  59.  93.  62.  40.  75.  94.  82.  46.  92.  79.  38.  62.
  30.  85.  89.  68.  42.  77.  79.  97.  82.  84.  61.  48.  69.  52.
  76.  39.  83.  67.  34.  33.  48.  99.  72.  72.  62.  99.  66.  81.
  94.  37.  70.  85.  69.  43.  50. 100.  45.  43.  51.  60.  84.  60.
  51.  95.  56.  90.  91.  89.  74.  51.  76.  87.  45.  67.  71.  38.
  85.  82.  46.  34.  36.  50.  75.  70.  57. 100.  63.  55.  89.  96.
  36.  82.  93.  84. 100.  90.  77.  80.  33.  52.  86.  48.  36.  42.
  75.  38.]


In [50]:
N = 100
M = 10
K = 3

x_op_res = np.zeros((N, M))
p_profit_gain = np.zeros(M)

i = 0
lambda_k = 1

for j in range(M):
    p_profit_gain[j] = p_profit[i][j] - np.sum(b_consumption[i][j] * lambda_k, axis=0)

print(p_profit_gain)

count = 0
for j in range(M):
    if p_profit_gain[j] > 0:
        print("count: %d, idx = %d, i=%d, j=%d, p_profit_gain =%f" % (count, (i * M + j), i, j, p_profit_gain[i * M + j]))
        count = count + 1

p_profit_gain = np.sort(p_profit_gain)[::-1]
print(p_profit_gain)

[  1. -11.  -3.   1. -14. -10. -12.  -2. -23. -13.]
count: 0, idx = 0, i=0, j=0, p_profit_gain =1.000000
count: 1, idx = 3, i=0, j=3, p_profit_gain =1.000000
[  1.   1.  -2.  -3. -10. -11. -12. -13. -14. -23.]


In [1]:
0.5 * (-6.426-20) + 0.5 * (-6.426) + 0.5 * (-6.426 - 2 * 20 + 0) + 30

-9.639000000000003